In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

response = WebBaseLoader(
    web_path="https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu",
    bs_kwargs=dict(parse_only=bs4.SoupStrainer("p"))
)
text = response.load()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
doc = splitter.split_documents(text)
doc

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu'}, page_content="Samantha Ruth Prabhu (born 28 April 1987) is an Indian actress who works predominantly in Telugu and Tamil films. One of South India's highest-paid actresses,[2][3][4] Samantha is the recipient of several accolades, including four Filmfare Awards South, two Nandi Awards and a Tamil Nadu State Film Award.\nAfter a brief stint as a model while pursuing a degree in commerce, Samantha made her acting debut in the Telugu romance film Ye Maaya Chesave (2010), winning the Filmfare Award for Best Female Debut – South. She became the second actress to win both the Filmfare Award for Best Actress – Tamil and Best Actress – Telugu in the same year, for her performances in the 2012 films Neethaane En Ponvasantham and Eega, respectively. Over the next few years, she played the leading lady in top-grossing androcentric films such as Dookudu (2011), Seethamma Vakitlo Sirimalle Chettu (2012), Attarintiki

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")
data_base = Chroma.from_documents(doc, embeddings)

In [8]:
query = data_base.similarity_search("samantha's awards")
query

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu'}, page_content='Samantha is the recipient of several awards, including four Filmfare Awards South, one Filmfare OTT Award, two Andhra Pradesh State Nandi Awards, four South Indian International Movie Awards, and three CineMAA Awards.'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu'}, page_content='Samantha is the recipient of several awards, including four Filmfare Awards South, one Filmfare OTT Award, two Andhra Pradesh State Nandi Awards, four South Indian International Movie Awards, and three CineMAA Awards.'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu'}, page_content='Her performance in A Aa (2016) won Samantha her fourth Filmfare Award, and she gained further praise for her performances in Mahanati (2018), Super Deluxe (2019), and Majili (2019), and the Amazon Prime Video thriller series The Family Man (2021). The last of t

In [9]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="deepseek-r1:1.5b")

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
1. You are an AI asisst, that will only answer from the given context,
2. you wont answer from your own knowladge except from the given context, 
3. make sure you elaborate and do no use your own knowladge,
5. if asked out-of-context, ans with "out of context"

<context>
{context}
<context>

Question:{input}
    """
)

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(model, prompt)
retriver = data_base.as_retriever()

In [14]:
from langchain.chains import create_retrieval_chain

re_chain = create_retrieval_chain(retriver, document_chain)
ans = re_chain.invoke({"input":"awards by sam"})
print(ans["answer"])

<think>
Okay, so I need to figure out how to answer this question about Samantha's awards based on the context provided. Let me start by reading through the information given.

First, I see that Samantha received several awards from different organizations. The context lists four Filmfare Awards South, one Filmfare OTT Award, two Andhra Pradesh State Nandi Awards, four South Indian International Movie Awards, and three CineMAA Awards. That's a total of 11 awards mentioned.

The user also provided additional information about her performances in specific movies: A Aa (2016) won her fourth Filmfare Award, Mahanati (2018), Super Deluxe (2019), and Majili (2019) earned her a Filmfare OTT Award. She starred in The Family Man (2021), which was the last of these awards she received. Then there were two failed movies, Aamnaandas (2024) and another one that I'm not sure about. After failing those, she starred in the action series Citadel: Honey Bunny (2024).

So, putting this together, Samantha

In [15]:
ans = re_chain.invoke({"input":"how many films did samantha do?"})
print(ans["answer"])

<think>
Okay, so I need to figure out how many films Samantha did. Let's see... From what I remember, there are these two main periods where Samantha was involved in big-budget hero movies. One of them is 2013 with "Jabardasth" and another from 2014 titled "Autonagar Surya." 

I think the first film she did was in 2013, called "Jabardasth," which had Pawan Kalyan to play. Then a few years later, she returned to the Tamil film industry with Manam in 2014, where she played both as a mother and a college student. 

Wait, does that mean each year she did two movies? Or was there only one per year? Hmm, I'm not sure. Let me check my notes again. The initial context mentioned "Samantha appeared increasingly in big-budget, hero-oriented films" starting from 2012. So maybe the first film was 2013 and then she did two more in 2014.

Putting that together, I think Samantha did three films in total: one in 2013 and two in 2014. That makes sense because the context provided the information about h

In [16]:
ans = re_chain.invoke({"input":"when is samanthas birthday"})
print(ans["answer"])

<think>
Okay, so I need to figure out Samantha's birthday. From what I know, she was born on April 28, 1987. She's from Chennai, Tamil Nadu, and her parents are a Telugu speaker and a Malayali speaker. She made her television debut in 2020, hosted Bigg Boss 4, then did some talk shows and showed episodes on Amazon Prime Video. That's all I remember about her.

Wait, the user asked specifically when her birthday is, so it's April 28th. They provided that information earlier in the context. But maybe they want more details or additional information? Let me check the context again to make sure there are no other birthdays mentioned.

Looking through the context, she was born on April 28, 1987, and her father is a Telugu speaker, her mother is a Malayali speaker. She was from Pallavaram neighborhood of Chennai, Tamil Nadu. So, all that's correct about her birthdate. There are no other birthdays mentioned beyond the birth date.

I think that's it. So, Samantha's birthday is on April 28th.
<